# Internship TASK 1

## <b>Importing all the library to be used

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,models
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

## Read train test csv file using pandas function "pd.read_csv"

In [ ]:
test=pd.read_csv('../input/gtsrb-german-traffic-sign/Test.csv')

In [ ]:
train=pd.read_csv('../input/gtsrb-german-traffic-sign/Train.csv')

### Checking the any missing, data distribution and description using Describe() function

In [ ]:
train.describe()

### Data Preprocessing : Resizing of image, changing it to array and preparing X_train

In [ ]:
X_train=[]
for i in range(len(train)):
    image = tf.keras.preprocessing.image.load_img('../input/gtsrb-german-traffic-sign/'+str(train['Path'][i]),grayscale=False, color_mode='rgb', target_size=(25,25),
    interpolation='nearest')
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])
    X_train.append(input_arr)

In [ ]:
###################### Another way to do above function using opencv ##################
# for i in range(len(train)):
#     img = cv2.imread('../input/gtsrb-german-traffic-sign/'+train['Path'][i], cv2.IMREAD_UNCHANGED)
#     train.Path[i]=cv2.resize(img, (25,25),interpolation = cv2.INTER_AREA)
# for i in range(len(train)):
#     X_train.append(np.array(train['Path'][i]))

##### copying the data for safety of my X_train data

In [ ]:
xtrain=X_train.copy()

In [ ]:
X_train=np.array(X_train)


In [ ]:
X_train.shape

#### Reshaping the data that is used as input shape in further step of model sequential

In [ ]:
X_train=X_train.reshape((39209,25, 25, 3))

#### Preparing the Y_train

In [ ]:
Y_train=train['ClassId']

#### Data Augmentaion of the images 
<font color=green> Data Augmentation of images just because to create accurate model by providing some more and data by doing its random contrast and random zoom of images

In [ ]:
data_augmentation=models.Sequential([
    layers.experimental.preprocessing.RandomContrast(0.85,input_shape=(25,25,3)),
    layers.experimental.preprocessing.RandomZoom(0.2)
])

### Creating the Convolution nueral network for my model for better predicting

In [ ]:
CNN=models.Sequential([
                      data_augmentation,
                       #cnn
                      layers.Conv2D(filters=20,kernel_size=(3,3),activation='relu'),
                      layers.MaxPooling2D((2,2)),
                       
                       layers.Conv2D(filters=40,kernel_size=(3,3),activation='relu'),
                       layers.MaxPooling2D((2,2)),
                        layers.Conv2D(filters=80,kernel_size=(3,3),activation='relu'),
                        layers.MaxPooling2D((2,2)),
                       #dense=
                       layers.Flatten(),
                      layers.Dense(400,activation='relu'),
                      layers.Dense(150,activation='relu'),
                      layers.Dense(43,activation='softmax')
])


##### Model Summary

In [ ]:
CNN.summary()

#### Model Fitting

In [ ]:
CNN.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
history=CNN.fit(X_train,Y_train,epochs=25)

#### Processing the X_test 

In [ ]:
X_test=[]
for i in range(len(test)):
    image = tf.keras.preprocessing.image.load_img('../input/gtsrb-german-traffic-sign/'+str(test['Path'][i]),grayscale=False, color_mode='rgb', target_size=(25,25),
    interpolation='nearest')
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])
    X_test.append(input_arr)

In [ ]:
xtest=X_test.copy()

In [ ]:
X_test=np.array(X_test)

In [ ]:
X_test.shape

In [ ]:
X_test=X_test.reshape(12630,25,25,3)

In [ ]:
test.head()

In [ ]:
Y_test=test.ClassId

### Evaluation of Model

In [ ]:
CNN.evaluate(X_test,Y_test)

In [ ]:
ypred=CNN.predict(X_test)

#### Model Accuracy Vs Epoch

In [ ]:

plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

#### Model Loss Vs Epoch

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

#### Prediction and Classificaiton Report

In [ ]:
y_pred_classes=[np.argmax(element) for element in ypred]
print("Classfication report\n",classification_report(Y_test,y_pred_classes))

#### Confusion Matrix

In [ ]:
plt.figure(figsize=(15,15))
sns.heatmap(confusion_matrix(Y_test,y_pred_classes))

### Saving the Model

In [ ]:
tf.keras.models.save_model(CNN,'Internship_task')

### Predictor of my model 

In [ ]:
model = tf.keras.models.load_model('Internship_task')

In [ ]:
def checker(img):
    img=np.array(img)
    img=cv2.resize(img,(25,25))
    img=np.array(img).reshape(-1,25,25,3)
    return np.argmax(model.predict(img))

In [ ]:
img=cv2.imread('../input/gtsrb-german-traffic-sign/Meta/1.png')

In [ ]:
checker(img)